# Microgrid Model


In [ ]:
county = "SanJoaquin"

### 1. Load Packages

In [ ]:
#import Pkg; Pkg.add("NBInclude"); #If there are any new packages to add later

In [1]:
using JuMP, Gurobi
using Plots; #plotly();
using VegaLite  
using DataFrames, CSV, PrettyTables
using FileIO
using Dates
using TimeZones
using NBInclude

### 2. Specify Directories

In [2]:
#inputs_dir = "/Users/laurenstreitmatter/Documents/Fourth_Year/ESC499/Inputs";
inputs_dir = "/home/lstreitmatter/Projects/thesis_runs/Inputs";
runs_df = CSV.read(joinpath(inputs_dir,"runs.csv"), DataFrame);
results_dir = "/home/lstreitmatter/Projects/thesis_runs/Results";
#results_dir = "/Users/laurenstreitmatter/Documents/Fourth_Year/ESC499/Results";

## Helper Functions

In [ ]:
function calculate_cap_annual(C, ror, n)
    return (C*(ror/(1-(1+ror)^(-n))))
end

In [ ]:
function create_df(DecisionVar,idcol::String,value_name::String)
    #DecisionVar = 2D decision variable indexed over an id and time
    #idcol = string with the id column name e.g., "h_id"
    #value_name = string with the name of the decision variable value e.g., "H2SOC"

    df = DataFrame(value.(DecisionVar).data, :auto)
    df_ax1 = value.(DecisionVar).axes[1]
    df_ax2 = value.(DecisionVar).axes[2]
    df_cols = names(df)
    insertcols!(df, 1, :idcol => df_ax1)
    df = stack(df, Not(:idcol), variable_name = :time)
    df.time = foldl(replace, [df_cols[i] => df_ax2[i] for i in 1:length(df_ax2)], init=df.time)
    rename!(df, :idcol => idcol)
    rename!(df, :value => value_name)
    df.time = convert.(DateTime,df.time)

    return df

end

In [ ]:
function create_df_one_axis(DecisionVar,value_name::String)
    df = DataFrame(value = value.(DecisionVar).data)  
    df_ax1 = value.(DecisionVar).axes[1]
    insertcols!(df, 1, :time => df_ax1) 
    df.time = convert.(DateTime,df.time)
    rename!(df, :value => value_name)
    
    return df
end

## Solver Function
Function to solve economic dispatch problem (over one year, one zone/node, deterministic)

In [ ]:
function renewable_microgrid(config, gen_df, solarcf_df, load_df, hydrogen_df, lmp_df, mer_df, battery_df)
    RM = Model(Gurobi.Optimizer)
    
    #Define Parameters from Config file 
    ror = parse(Float32,config[config[!,:Parameter] .== "ror", :Value][1])
    carbon_cap = parse(Float32,config[config[!,:Parameter] .== "carbon_cap", :Value][1])
    carbon_cost = parse(Float32,config[config[!,:Parameter] .== "carbon_cost", :Value][1])/1000
    voll = parse(Float32,config[config[!,:Parameter] .== "voll", :Value][1])
    county_print = config[config[!,:Parameter] .== "county", :Value][1]
    rps = parse(Float32,config[config[!,:Parameter] .== "rps", :Value][1])

    #Define sets 
        #A set of all renewable generators
    G_renew = gen_df[gen_df[!,:renewable_gen] .== 1,:type]
        #A set of non-renewable generators
    G_fossil = gen_df[gen_df[!,:renewable_gen] .== 0,:type]
        #A set of all generators
    G = gen_df.type

        #A set of the timepoints (most granular time input)
    TP = load_df.time
        #Variable for the amount of time between timepoints, in minutes
    tp_spacing = convert(Dates.Minute,TP[2]-TP[1])
        #Integer version of tp_spacing converted to hours
    tp_spacing_int = convert(Float32,tp_spacing.value)/60

    #Define outage timepoints when Pgrid must be zero
    outage_length = parse(Float32,config[config[!,:Parameter] .== "outage_duration", :Value][1])
    outage_start = config[config[!,:Parameter] .== "outage_start", :Value][1]
    temp_start = DateTime(outage_start,dateformat"yyyy-mm-dd HH:MM:SS+zz") + Dates.Year(1) #offset to 2021 to avoid leap year
    temp_end = temp_start + Dates.Day(outage_length) - tp_spacing
    tp_outage = temp_start:tp_spacing:temp_end;
    tp_outage = tp_outage .- Dates.Year(1); #reset back to 2020
    println("here2")

    println("here3")
    # Capacity building/sizing decision variables
    @variables(RM, begin
        0 <=    vGenCapkW[G]        
        0 <=    vBatteryCapkW       <= battery_df.max_storage_kw[1]
        0 <=    vElectroCapkW       <= hydrogen_df.electrolyzer_max_capacity_kw[1]
        0 <=    vFCCapkW            <= hydrogen_df.fuel_cell_max_capacity_kw[1]
        0 <=    vTankCapkg          <= hydrogen_df.tank_max_capacity_kg[1]
    end)

    # Operational decision variables   
    @variables(RM, begin
        0 <=    vGEN[G, TP]         # power generation/production (kW)

        0 <=    vELECTROLYZER[TP]   # electricity assigned to hydrogen production (kW)
        0 <=    vH2SOC[TP]          # hydrogen state of charge (kg H2)
        0 <=    vFUELCELL[TP]       # fuel cell electricity production (kW)

        0 <=    vCharge[TP]         # amount to charge battery (kW)
        0 <=    vBSOC[TP]           # battery state of charge (kWh)
        0 <=    vDischarge[TP]      # amount to discharge battery (kW)

        0 <=    vPLoss[TP]          # unmet load (kW)

        vGrid[TP]                   # positive vGrid means consuming energy from the grid

        
    end)

    #Define another variable to be used for carbon cost calculations (only want vGrid when using grid elec)
    @variable(RM, vPosGrid[TP] >= 0)
    @constraint(RM, cPosGrid[tp in TP], vPosGrid[tp] >= vGrid[tp])

    #Define cost expressions to make code cleaner
    #grid operating cost
    @expression(RM, grid_op_cost, sum(lmp_df[lmp_df.time .== tp,:lmp][1] * vGrid[tp] * tp_spacing_int for tp in TP))

    # hydrogen costs
    @expression(RM, elect_cap_cost, calculate_cap_annual(hydrogen_df.electrolyzer_cap_cost_per_kw[1] * vElectroCapkW,ror,
                                                            hydrogen_df.electrolyzer_lifetime[1]))
    @expression(RM, fc_cap_cost, calculate_cap_annual(hydrogen_df.fuel_cell_cap_cost_per_kw[1] * vFCCapkW, ror,
                                                            hydrogen_df.fuelcell_lifetime[1]))
    @expression(RM, tank_cap_cost, calculate_cap_annual(hydrogen_df.tank_cap_cost_per_kg[1] * vTankCapkg, ror,
                                                            hydrogen_df.tank_lifetime[1]))                                                  
    @expression(RM, elect_fom_cost, hydrogen_df.electrolyzer_fom_cost_per_kw[1] * vElectroCapkW)
    @expression(RM, fc_fom_cost, hydrogen_df.fuel_cell_fom_cost_per_kw[1] * vFCCapkW)

    # battery costs
    @expression(RM, batt_cap_cost, calculate_cap_annual(battery_df.cap_cost_per_kw[1] * vBatteryCapkW, ror,
                                                            battery_df.lifetime[1]))
    @expression(RM, batt_fom_cost, battery_df.fom_cost_per_kw[1] * vBatteryCapkW) 
    
    # EJ cost metrics 
    @expression(RM, tot_lost_load, sum(vPLoss[tp] * tp_spacing_int for tp in TP))
    @expression(RM, reliability_cost, tot_lost_load * voll)   
    @expression(RM, diesel_co2, sum(vGEN["diesel",tp] * tp_spacing_int for tp in TP) * 
                                    gen_df[gen_df.type .== "diesel",:carbon_kg_per_kwh][1])
    @expression(RM, net_grid_co2, sum(mer_df[mer_df.time .== (tp - Minute(tp)),:mer_kg_kWh][1] * vGrid[tp] * tp_spacing_int
                                        for tp in TP))
    @expression(RM, tot_grid_co2, sum(mer_df[mer_df.time .== (tp - Minute(tp)),:mer_kg_kWh][1] * vPosGrid[tp] * tp_spacing_int
                                        for tp in TP))
    @expression(RM, diesel_carb_cost, diesel_co2 * carbon_cost)
    @expression(RM, tot_grid_carb_cost, tot_grid_co2 * carbon_cost)    
    @expression(RM, tot_carb_cost, diesel_carb_cost + tot_grid_carb_cost)
    
    # Diesel costs
    @expression(RM, diesel_cap_cost, calculate_cap_annual(gen_df[gen_df.type .== "diesel", :cap_cost_per_kw][1] * vGenCapkW["diesel"], 
                            ror,gen_df[gen_df.type .== "diesel",:lifetime][1]))
    @expression(RM, diesel_fom_cost, gen_df[gen_df.type .== "diesel", :fom_per_kw][1] * vGenCapkW["diesel"])
    @expression(RM, diesel_vom_cost, sum(gen_df[gen_df.type .== "diesel", :om_cost_per_kwh][1] * vGEN["diesel",tp] * 
                            tp_spacing_int for tp in TP))
    @expression(RM, diesel_fuel_cost, sum(gen_df[gen_df.type .== "diesel", :fuel_cost_per_kwh][1] * vGEN["diesel",tp] * 
                            tp_spacing_int for tp in TP))
    # Solar costs 
    @expression(RM, solar_cap_cost, calculate_cap_annual(gen_df[gen_df.type .== "solar", :cap_cost_per_kw][1] * vGenCapkW["solar"], 
                            ror,gen_df[gen_df.type .== "solar",:lifetime][1]))
    @expression(RM, solar_fom_cost, gen_df[gen_df.type .== "solar", :fom_per_kw][1] * vGenCapkW["solar"])
    @expression(RM, solar_vom_cost, sum(gen_df[gen_df.type .== "solar", :om_cost_per_kwh][1] * vGEN["solar",tp] * 
                            tp_spacing_int for tp in TP))
    @expression(RM, solar_fuel_cost, sum(gen_df[gen_df.type .== "solar", :fuel_cost_per_kwh][1] * vGEN["solar",tp] * 
                            tp_spacing_int for tp in TP))

    println("here4")
    #Objective Function
    @objective(RM, Min, 
        #diesel costs (capital, fixed o&m, variable o&m, fuel)
        diesel_cap_cost + diesel_fom_cost + diesel_vom_cost + diesel_fuel_cost +
        #solar costs (capital, fixed o&m, variable o&m, fuel)
        solar_cap_cost + solar_fom_cost + solar_vom_cost + solar_fuel_cost +  
        #grid costs (fuel) 
        grid_op_cost + 
        #hydrogen costs (capital and o&m - no tank o&m)
        elect_cap_cost + fc_cap_cost + tank_cap_cost + elect_fom_cost + fc_fom_cost +
        #battery costs (capital and o&m)
        batt_cap_cost + batt_fom_cost +  
        # EJ METRICS
        # Reliability Cost (Loss of load) and Carbon Cost - Grid and Diesel 
        reliability_cost + tot_carb_cost
    )
    println("here5")
    #Constraints
    #Supply-Demand Balance Constraints
    @constraint(RM, cDemand[tp in TP], 
        sum(vGEN[g,tp] for g in G) +
        vGrid[tp] +
        vPLoss[tp] - 
        vELECTROLYZER[tp] + 
        vFUELCELL[tp] -
        vCharge[tp] +
        vDischarge[tp] -  
        load_df[load_df.time .== tp,:loadkW][1] == 0
    );
    println("here6")

    # Grid OUTAGE constraint (if during an outage, no net electricity to/from grid)
    @constraint(RM, cGridOutage[tp in tp_outage], vGrid[tp] == 0)

    # Generator Capacity constraints 
    @constraint(RM, cCap_dieselBuild, vGenCapkW["diesel"] <= gen_df[gen_df.type .== "diesel",:max_capacity_kw][1])
    @constraint(RM, cCap_diesel[tp in TP], 
            vGEN["diesel", tp] <= 
                    vGenCapkW["diesel"])

    # Renewable Generation Capacity Constraints
    @constraint(RM, cCap_solarBuild, vGenCapkW["solar"] <= gen_df[gen_df.type .== "solar",:max_capacity_kw][1])
    @constraint(RM, cCap_solar[tp in TP], 
            vGEN["solar", tp] <= 
                    vGenCapkW["solar"] * solarcf_df[solarcf_df.time .== tp,:cf][1])

    #RPS Constraint (use instead of carbon cap)
    @constraint(RM, cRPS,
                    sum(vGEN["solar",tp] * tp_spacing_int for tp in TP) >= 
                    rps * sum(load_df[load_df.time .== tp,:loadkW][1] * tp_spacing_int for tp in TP))
       

    
    println("here7")
    #CARBON CAP (Annual Cap)
    @constraint(RM, cCarbCap, 
                    sum(vGEN["diesel",tp] * tp_spacing_int  * gen_df[gen_df.type .== "diesel",:carbon_kg_per_kwh][1] 
                        for tp in TP) +
                    sum(vGrid[tp] * tp_spacing_int * mer_df[mer_df.time .== (tp - Minute(tp)),:mer_kg_kWh][1]
                        for tp in TP)
                    <= carbon_cap)
    
    println("here8")
    #First Timepoint for storage initialization
    tp1 = minimum(load_df.time)
    tp_end = maximum(load_df.time)
    TP_SOC = setdiff(TP, [tp1])
    #HYDROGEN CONSTRAINTS
    @constraints(RM, begin
    # Electrolyzer, fuel cell, and tank capacity constraints taken care of in build variables
    # Electrolyzer, fuel cell, and tank operational constraints 
        cElectroOpCap[tp in TP], vELECTROLYZER[tp]  <= vElectroCapkW
        cFCOpCap[tp in TP], vFUELCELL[tp]  <= vFCCapkW
        cTankOpCap[tp in TP], vH2SOC[tp]  <= vTankCapkg

    # (1) Hydrogen storage tank state of charge, balance in each timepoint
        cH2SOC[tp in TP_SOC], 
        vH2SOC[tp] == vH2SOC[(tp + Dates.Year(1)) - tp_spacing - Dates.Year(1)] + 
                        (hydrogen_df.electrolyzer_kg_per_kwh[1])*vELECTROLYZER[tp] * tp_spacing_int - 
                        vFUELCELL[tp] * tp_spacing_int/(hydrogen_df.fuel_cell_kwh_per_kg[1]) 

    # (4) Set initial and final storage to half capacity
        vH2SOC[tp1] == 0.5 * vTankCapkg + 
                        (hydrogen_df.electrolyzer_kg_per_kwh[1])*vELECTROLYZER[tp1] * tp_spacing_int - 
                        vFUELCELL[tp1] * tp_spacing_int/(hydrogen_df.fuel_cell_kwh_per_kg[1]) 
        vH2SOC[tp_end] == 0.5 * vTankCapkg

    end)

    #Battery CONSTRAINTS
    @constraints(RM, begin
    # Battery storage build constraints taken care of in decision variables
    # Operational constraints (1) - (3)
    cBatteryOpChargeCap[tp in TP], vCharge[tp]  <= vBatteryCapkW 
    cBatteryOpDischargeCap[tp in TP], vDischarge[tp]  <= vBatteryCapkW 
    cBatteryOpStorCap[tp in TP], vBSOC[tp]  <= vBatteryCapkW * battery_df.energy_power_ratio[1]
    
    # (4) Set initial and final storage to half capacity 
    vBSOC[tp1] == 0.5 * vBatteryCapkW * battery_df.energy_power_ratio[1] + 
                    battery_df.charge_efficiency[1]*vCharge[tp1]*tp_spacing_int - 
                    vDischarge[tp1]*tp_spacing_int/(battery_df.discharge_efficiency[1])
    vBSOC[tp_end] == 0.5 * vBatteryCapkW * battery_df.energy_power_ratio[1]
  
    # (5) Battery storage state of charge, balance in each timepoint
    cBSOC[tp in TP_SOC], 
        vBSOC[tp] == vBSOC[(tp + Dates.Year(1)) - tp_spacing - Dates.Year(1)] + 
                        battery_df.charge_efficiency[1]*vCharge[tp]*tp_spacing_int - 
                        vDischarge[tp]*tp_spacing_int/(battery_df.discharge_efficiency[1])
    
    # (6) Limit discharge per year
    cBatteryTotDischarge,
        sum(vDischarge[tp] for tp in TP) * tp_spacing_int / (battery_df.discharge_efficiency[1]) <=
            vBatteryCapkW * battery_df.energy_power_ratio[1] * battery_df.cycles_per_year[1]

    end)

    #Some final expressions for printing results more easily

    #Total Energy Use results
    @expression(RM, tot_kwh_solar, sum(vGEN["solar",tp] * tp_spacing_int for tp in TP))
    @expression(RM, tot_kwh_diesel, sum(vGEN["diesel",tp] * tp_spacing_int for tp in TP))
    @expression(RM, net_kwh_grid, sum(vGrid[tp] * tp_spacing_int for tp in TP)) #net energy to/from grid
    @expression(RM, pos_kwh_grid, sum(vPosGrid[tp] * tp_spacing_int for tp in TP)) #total energy taken from grid
    @expression(RM, tot_kwh_elect, -1 * sum(vELECTROLYZER[tp] * tp_spacing_int for tp in TP))
    @expression(RM, tot_kwh_fc, sum(vFUELCELL[tp] * tp_spacing_int for tp in TP))
    @expression(RM, tot_kwh_battcharge, -1 * sum(vCharge[tp] * tp_spacing_int for tp in TP))
    @expression(RM, tot_kwh_battdischarge, sum(vDischarge[tp] * tp_spacing_int for tp in TP))

    #Total load to cover during outage
    @expression(RM, tot_load_outage, sum(load_df[load_df.time .== tp,:loadkW][1] * tp_spacing_int for tp in tp_outage))


    # Solve statement (! indicates runs in place)
    optimize!(RM)

    # RESULTS
    # Dataframe of optimal decision variables - solar and diesel generation
    gen_solution = create_df(vGEN,"type","Gen(kW)");
    gen_solution = unstack(gen_solution, :time, :type, :"Gen(kW)")
    grid_solution = create_df_one_axis(vGrid,"Grid(kW)")
    #Load loss solution
    ploss_solution = create_df_one_axis(vPLoss, "PLoss(kW)")
    #Load info for reference
    load_info = load_df[:,[:time,:loadkW]]
    #Hydrogen results
    hydrogen_solution = create_df_one_axis(vH2SOC,"H2SOC(kg)");
    elect_solution = create_df_one_axis(vELECTROLYZER,"ElectrolyzerRun(kW)");
    fc_solution = create_df_one_axis(vFUELCELL,"FuelCellRun(kW)");
    #Battery results
    battery_solution = create_df_one_axis(vBSOC,"BSOC(kWh)");
    bcharge_solution = create_df_one_axis(vCharge,"ChargeBattery(kW)");
    bdischarge_solution = create_df_one_axis(vDischarge,"DischargeBattery(kW)");

    #Combine all variables into one dataframe
    gen_solution = innerjoin(gen_solution, grid_solution,elect_solution, hydrogen_solution, fc_solution, bcharge_solution, battery_solution, 
                                bdischarge_solution,ploss_solution,load_info, on = :time)

    #Total Load (kWh)
    total_load = sum(load_df[load_df.time .== tp,:loadkW][1] * tp_spacing_int for tp in TP);

    # RESULTS FOR REPORT USE HERE
    report_results = DataFrame(Category = String[], Metric = String[], Value = Float32[], Unit = String[])

    #Add objective costs
    push!(report_results, ["Cost"  "Total Cost" objective_value(RM) "\$"])
    push!(report_results, ["Cost"  "Solar Capital Cost" value(solar_cap_cost) "\$"])
    push!(report_results, ["Cost"  "Diesel Capital Cost" value(diesel_cap_cost) "\$"])
    push!(report_results, ["Cost"  "Electrolyzer Capital Cost" value(elect_cap_cost) "\$"])
    push!(report_results, ["Cost"  "Fuel Cell Capital Cost" value(fc_cap_cost) "\$"])
    push!(report_results, ["Cost"  "Tank Capital Cost" value(tank_cap_cost) "\$"])
    push!(report_results, ["Cost"  "Battery Capital Cost" value(batt_cap_cost) "\$"])

    investment_cost = value(solar_cap_cost) + value(diesel_cap_cost) + value(elect_cap_cost) + 
                        value(fc_cap_cost) + value(tank_cap_cost) + value(batt_cap_cost)
    push!(report_results, ["Cost" "Total Annualized Investment Cost" investment_cost "\$"])

    push!(report_results, ["Cost"  "Solar FOM Cost" value(solar_fom_cost) "\$"])
    push!(report_results, ["Cost"  "Diesel FOM Cost" value(diesel_fom_cost) "\$"])
    push!(report_results, ["Cost"  "Diesel Fuel Cost" value(diesel_fuel_cost) "\$"])
    push!(report_results, ["Cost"  "Grid Electricity Cost" value(grid_op_cost) "\$"])
    push!(report_results, ["Cost"  "Electrolyzer FOM Cost" value(elect_fom_cost) "\$"])
    push!(report_results, ["Cost"  "Fuel Cell FOM Cost" value(fc_fom_cost) "\$"])
    push!(report_results, ["Cost"  "Battery FOM Cost" value(batt_fom_cost) "\$"]) 

    operation_cost = value(solar_fom_cost) + value(diesel_fom_cost) + value(diesel_fuel_cost) + 
                        value(grid_op_cost) + value(elect_fom_cost) + value(fc_fom_cost) + 
                        value(batt_fom_cost)
    push!(report_results, ["Cost" "Total Annual Operation Cost" operation_cost "\$"])

    push!(report_results, ["Cost"  "Lost Load Cost" value(reliability_cost) "\$"])
    push!(report_results, ["Cost"  "Carbon Cost" value(tot_carb_cost) "\$"])

    ej_cost = value(reliability_cost) + value(tot_carb_cost)
    push!(report_results, ["Cost"  "EJ Cost" ej_cost "\$"])

    #Add EJ Metrics
    push!(report_results, ["EJ"  "Total Lost Load" value(tot_lost_load) "kWh"])
    push!(report_results, ["EJ"  "Diesel Emissions" value(diesel_co2) "kg CO2"])
    push!(report_results, ["EJ"  "Net Grid Emissions" value(net_grid_co2) "kg CO2"])
    push!(report_results, ["EJ"  "Total Grid Emissions" value(tot_grid_co2) "kg CO2"])
    tot_co2 = value(diesel_co2) + value(tot_grid_co2)
    net_co2 = value(diesel_co2) + value(net_grid_co2)
    push!(report_results, ["EJ"  "Total Emissions" tot_co2 "kg CO2"])
    push!(report_results, ["EJ"  "Net Emissions" net_co2 "kg CO2"])

    #Add Build Capacities
    push!(report_results, ["Capacity"  "Solar" value(vGenCapkW["solar"]) "kW"]) 
    push!(report_results, ["Capacity"  "Diesel" value(vGenCapkW["diesel"]) "kW"]) 
    push!(report_results, ["Capacity"  "Electrolyzer" value(vElectroCapkW) "kW"]) 
    push!(report_results, ["Capacity"  "Fuel Cell" value(vFCCapkW) "kW"]) 
    push!(report_results, ["Capacity"  "Tank" value(vTankCapkg) "kg H2"]) 
    push!(report_results, ["Capacity"  "Battery (4h)" value(vBatteryCapkW) "kW"])

    #Add Energy Use
    push!(report_results, ["Energy Output"  "Solar" value(tot_kwh_solar) "kWh"])
    push!(report_results, ["Energy Output"  "Diesel" value(tot_kwh_diesel) "kWh"])
    push!(report_results, ["Energy Output"  "Net Grid" value(net_kwh_grid) "kWh"])
    push!(report_results, ["Energy Output"  "Positive Grid" value(pos_kwh_grid) "kWh"])
    push!(report_results, ["Energy Output"  "Fuel Cell" value(tot_kwh_fc) "kWh"])
    push!(report_results, ["Energy Output"  "Electrolyzer" value(tot_kwh_elect) "kWh"])
    push!(report_results, ["Energy Output"  "Battery Discharge" value(tot_kwh_battdischarge) "kWh"])
    push!(report_results, ["Energy Output"  "Battery Charge" value(tot_kwh_battcharge) "kWh"])
    push!(report_results, ["Energy Output"  "Annual Load" value(total_load) "kWh"])
    push!(report_results, ["Energy Output"  "Total Load during Outage" value(tot_load_outage) "kWh"])


    #Add Relevant Inputs
    push!(report_results, ["Input"  "Carbon Cost" carbon_cost "\$/kg CO2"])
    push!(report_results, ["Input"  "VoLL" voll "\$/kWh"])
    push!(report_results, ["Input"  "Hydrogen Electrolyzer Cost" hydrogen_df.electrolyzer_cap_cost_per_kw[1] "\$/kW"])
    push!(report_results, ["Input"  "Outage Length" outage_length "Days"])
    push!(report_results, ["Input"  outage_start 0.0 "Outage Start Date"])
    push!(report_results, ["Input"  "RPS" rps "None"])
    # END OF RESULTS FOR REPORT USE

    # Return the solution and objective as named tuple
    return (
        gen_solution = gen_solution, #includes all timestamped variables
        #cost = objective_value(RM),
        status = termination_status(RM),
        report_results = report_results
    )

end

## 3. Loop through Runs

In [7]:
for row in eachrow(runs_df)
    run = row.runs 
    #Load Config File
    config_filename = string("config_", county, "_", run, ".csv")
    datadir = joinpath(inputs_dir,"ConfigFiles",county,config_filename)
    config = CSV.read(datadir, DataFrame);


    #Load files from Config inputs
    gen_filename = config[config[!,:Parameter] .== "generator_build_file", :Value][1]
    solar_cf_filename = config[config[!,:Parameter] .== "solar_cf_file", :Value][1]
    lmp_filename = config[config[!,:Parameter] .== "lmp_file", :Value][1]
    grid_mer_filename = config[config[!,:Parameter] .== "grid_mer_file", :Value][1]
    load_filename = config[config[!,:Parameter] .== "load_file", :Value][1]
    load_setting = config[config[!,:Parameter] .== "load_setting", :Value][1]
    hydrogen_filename = config[config[!,:Parameter] .== "hydrogen_file", :Value][1]
    battery_filename = config[config[!,:Parameter] .== "battery_file", :Value][1]
    load_setting = config[config[!,:Parameter] .== "load_setting", :Value][1]

    #Load Generator Build Inputs
    datadir = joinpath(inputs_dir,"BuildInputs",gen_filename); 
    gen_df = CSV.read(datadir, DataFrame);

    #Load Solar CF Inputs
    datadir = joinpath(inputs_dir,"SolarProfiles",solar_cf_filename); 
    solarcf_df = CSV.read(datadir, DataFrame);
    #Convert solar cf time column to datetime format
    solarcf_df.time = DateTime.(solarcf_df.time,dateformat"y-m-d HH:MM:S+zz");

    #Load LMP Inputs
    datadir = joinpath(inputs_dir,"LMPs",lmp_filename);  
    lmp_df = CSV.read(datadir, DataFrame);
    #Convert time column to datetime format
    lmp_df.time = String.(lmp_df.time)
    tzdf = Dates.DateFormat("yyyy-mm-ddTH:M:S-zz")
    lmp_df.time = DateTime.(ZonedDateTime.(lmp_df.time,tzdf));
    #Convert lmp column to $/kWh from $/MWh
    lmp_df.lmp = lmp_df.lmp./1000;
    #Remove feb 29 (leap year) from lmp data
    lmp_df = filter(:time => x -> Date(x) != Dates.Date("2020-02-29"), lmp_df);

    #Load MER inputs
    datadir = joinpath(inputs_dir,"BuildInputs",grid_mer_filename);  
    mer_df = CSV.read(datadir, DataFrame);
    #Convert time column to datetime format
    mer_df.time = String.(mer_df.time)
    tzdf = Dates.DateFormat("yyyy-mm-dd H:M:S+zz")
    mer_df.time = DateTime.(ZonedDateTime.(mer_df.time,tzdf));
    #Remove feb 29 (leap year) from mer data
    mer_df = filter(:time => x -> Date(x) != Dates.Date("2020-02-29"), mer_df);

    #Load load profiles
    datadir = joinpath(inputs_dir,"LoadProfiles",load_setting,load_filename);
    load_df = CSV.read(datadir, DataFrame);
    #Convert load time column to datetime format
    load_df.time = DateTime.(load_df.time,dateformat"yyyy-mm-dd HH:MM:SS+zz");

    #Check timestamps align
    println(sort(lmp_df.time) == solarcf_df.time)
    println(solarcf_df.time == load_df.time)

    #Load hydrogen inputs
    datadir = joinpath(inputs_dir,"BuildInputs",hydrogen_filename);  
    hydrogen_df = CSV.read(datadir, DataFrame);

    #Load battery inputs
    datadir = joinpath(inputs_dir,"BuildInputs",battery_filename);  
    battery_df = CSV.read(datadir,DataFrame);

    #RUN SOLVER
    solution = renewable_microgrid(config, gen_df, solarcf_df, load_df, hydrogen_df, lmp_df, mer_df, battery_df);

    #PRINT OUTPUTS
    gen_solution = solution[:gen_solution];
    report_solution = solution[:report_results];
    datadir = joinpath(results_dir,county,string(run, "_genresults.csv"));
    CSV.write(datadir,gen_solution);
    datadir = joinpath(results_dir,county,string(run,"_reportresults.csv"));
    CSV.write(datadir,report_solution);

    println(string(county, run, "_outputs completed"))


end


config_SanJoaquin_carb25.csv
config_SanJoaquin_carb50.csv
config_SanJoaquin_carb100.csv
config_SanJoaquin_carb200.csv


#### Load Solar Capacity Factors